<a href="https://colab.research.google.com/github/RafsanJany-44/SIgnal_Processing/blob/main/Signal_Processing_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 21.3 MB/s eta 0:00:00


In [ ]:
from glob import glob
import os
import mne
import pandas
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data_path = "/content/drive/MyDrive/Signal_Data/dataverse_files"

In [ ]:
all_file_path = glob(data_path+"/*.edf")
print(len(all_file_path))

28


In [ ]:
all_file_path

['/content/drive/MyDrive/Signal_Data/dataverse_files/h12.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h05.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h06.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h04.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h08.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h09.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h11.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h02.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h07.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h03.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h01.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h10.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/s01.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/s07.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h13.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h1

In [ ]:
healthy_file_path = []
patient_file_path = []

for i in all_file_path:
  if str(os.path.basename(i))[0] == 'h':
    healthy_file_path.append(i)
  else:
    patient_file_path.append(i)

In [ ]:
def read_data(file_path):
  data = mne.io.read_raw_edf(file_path, preload=True)
  data.set_eeg_reference()
  data.filter(l_freq=0.5,h_freq=45)
  epochs=mne.make_fixed_length_epochs(data,duration=5,overlap=1)
  array=epochs.get_data()
  return array

In [ ]:

sample_data = read_data(healthy_file_path[0])

Extracting EDF parameters from /content/drive/MyDrive/Signal_Data/dataverse_files/h12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 224999  =      0.000 ...   899.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (6.604 s)

Not setting metadata
224 matching events found
No baseline correction applied
0 projection i

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


In [ ]:
sample_data.shape   #epochs,channels, length of signals

(224, 19, 1250)

In [ ]:
%%capture
control_epochs_array = []
patient_epochs_array = []

for i in healthy_file_path:
  control_epochs_array.append(read_data(i))

for i in patient_file_path:
  patient_epochs_array.append(read_data(i))

In [ ]:
control_epochs_array[0].shape

(224, 19, 1250)

In [ ]:
patient_epochs_array[0].shape

(211, 19, 1250)

In [ ]:

#labeling control as 0. -> Unhealthy
control_epochs_label = []
for i in control_epochs_array:
  control_epochs_label.append(len(i)*[0])


#labeling healthy as 1. -> Unhealthy
patient_epochs_label = []
for i in patient_epochs_array:
  patient_epochs_label.append(len(i)*[1])

In [ ]:
data_list=control_epochs_array+patient_epochs_array
label_list=control_epochs_label+patient_epochs_label

In [ ]:
#spliting data as subject, makig group for each subject

group_list = []


for i,j in enumerate(data_list):
  group_list.append([i]*len(j))

len(group_list)




28

In [ ]:
data_array = np.vstack(data_list)
label_array = np.hstack(label_list)
group_array = np.hstack(group_list)
data_array.shape,label_array.shape,group_array.shape

((7201, 19, 1250), (7201,), (7201,))

In [ ]:
from scipy import stats

def mean(x):
  return np.mean(x,axis= -1)

def std(x):
  return np.std(x,axis= -1)

def ptp(x):
  return np.ptp(x,axis= -1)

def minim(x):
  return np.min(x,axis= -1)

def maxim(x):
  return np.max(x,axis= -1)

def argminim(x):
  return np.argmin(x,axis= -1)

def argmaxim(x):
  return np.argmax(x,axis= -1)

def rms(x):
  return np.sqrt(np.mean(x**2,axis=-1))

def abs_diff_signal(x):
  return np.sum(np.abs(np.diff(x,axis=-1)),axis=-1)

def skewness(x):
  return stats.skew(x,axis=-1)

def kurtosis(x):
  return stats.kurtosis(x,axis = -1)

def concatenate_features(x):
  return  np.concatenate((mean(x),std(x),ptp(x),minim(x),maxim(x),argminim(x),argmaxim(x), rms(x),abs_diff_signal(x),skewness(x),kurtosis(x)),axis=-1)

In [ ]:
features = []
for d in data_array:
  feattures.append(concatenate_features(d))

In [ ]:
features_array = np.array(feattures)
feattures_array.shape

(14402, 209)

In [63]:
epochs_array=control_epochs_array+patient_epochs_array
epochs_label=control_epochs_label+patient_epochs_label

In [64]:
epochs_array = np.vstack(epochs_array)
epochs_label = np.vstack(epochs_label)
groups_array = np.hstack(group)

ValueError: ignored